In [158]:
import pandas as pd
import spacy
from spacy import displacy
filename='---' #name of file containing transcript

In [159]:
#Function read the text file into a dataframe named "df"
#Two Column are made named "Speaker" , " Dialogue"
def txt_to_df (filename):
   df=pd.read_csv(filename,sep=':',names=["Speaker","Dialogue"])
   return df

In [161]:
# this function extract all the name and entity from the "Dialogue" Column of the dataframe and store them in a list called 'ner'
def extract_ner(df):
  NER = spacy.load("en_core_web_sm",disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"])
  ner=[]
  for i in range(0,len(df)):
    text=NER(df["Dialogue"][i])
    ner.append(list(text.ents))
  return ner

In [162]:
# Extract the only Speaker names and store them in list called Speaker_names 
def extract_speaker(names):
  for d in names:
    for word in d[:]:
      if(word.label_!="PERSON"):
          d.remove(word)
  return names

In [163]:
# a Entity Column was added in dataframe which hold the speaker name of desired utterances
def add_name_dataframe (df,names):
  entity=[]
  for n in names:
    entity.append(n)
  df["Entity"]=entity
  return df

In [164]:
# a function is to check the list contain item or not
def find_item(list, item):
  for i in range(0,len(list)):
      if (list[i]==item):
        return True
  
  return False

In [165]:
# Replace function will replace the speaker name with their correct name in dataframe
def replace(df,Occur):
  for i in range(0,len(df)):
    if len(df["Entity"][i])>1:
      for j in range(0,len(df["Entity"][i])):
         if(find_item(Occur,str(df["Entity"][i][j])))!=True:
          if (find_item(Occur,str(df["Speaker"][i]))!= True):
              df["Speaker"]=df["Speaker"].replace([df['Speaker'][i]],[str(df["Entity"][i][j])])
              Occur.append(str(df["Entity"][i][j]))
    elif (len(df["Entity"][i])==1 and (not(find_item (Occur,str(df["Entity"][i][0]))))):
      if (find_item(Occur,str(df["Speaker"][i]))!= True):
        Occur.append(str(df["Entity"][i][0]))
        df["Speaker"]=df["Speaker"].replace([df['Speaker'][i]],[str(df["Entity"][i][0])])
  df.drop('Entity', inplace=True, axis=1)
  return df

In [173]:
# function update the transcript file with speaker name
def df_to_txt(df,filename):
  with open(filename,'w') as file:
    pass
  df.to_csv(r'transcript.txt', header=None, index=None, sep=':', mode='a')

In [174]:
Occur=[] # it hold the speaker name which is already assigned 
df=txt_to_df(filename)
ner=extract_ner(df)
Speaker_names=extract_speaker(ner)
df=add_name_dataframe (df,Speaker_names)
df=replace(df,Occur)
df_to_txt(df,filename)


In [175]:
df

,Speaker,Dialogue
0,Alice Roberts,"Hello, I’d like to introduce myself. My name ..."
1,David Smith,"Hi, I’m David Smith and I work in book sales ..."
2,Sally,"Thanks David. So as David said, I’m Sally, Sa..."
3,Paul Owens,"Nice to meet everyone, although I already kno..."
4,Alice Roberts,"Thanks everyone, and yes just to bring David ..."
5,Sally,"Ah, that explains why we’re here!"
6,David Smith,Yes.
